In [8]:
from openforcefield.topology import Molecule
from openforcefield.typing.engines.smirnoff import ForceField
from simtk.openmm.app import PDBFile 

In [11]:
mol = Molecule.from_smiles('CC')
mol2_file = Molecule.from_file('ethane.sdf')